In [112]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.corpus import stopwords
import nltk


In [113]:
data=pd.read_json('/Users/liujingyi/Desktop/News_Category_Dataset_v3.json',lines=True)
data=data.dropna()
data

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [114]:
df = data.sort_values(by='date') 
train_size=0.7
split_index = int(len(df) * train_size)
train = df[:split_index]
test= df[split_index:]
# train = df[0:7000]
# test= df[7000:10000]

In [115]:
def preprocess(text):
    text = text.lower()  # 转换为小写
    text = re.sub(r'\b\d+\b', '', text)  # 移除数字
    text = re.sub(r'[^a-z\s]', '', text)  # 移除标点
    words = text.split()  # 分词
    words = [word for word in words if word not in stopwords.words('english')]  # 移除停用词
    return ' '.join(words)

train['processed'] = train['short_description'].apply(preprocess)
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(train['processed'])
vocab = vectorizer.get_feature_names_out()

# 获取文档-词矩阵
doc_word_matrix = X.toarray()

# 转换为 onlineldavb.py 可接受的格式
D = len(train)  # 文档总数
W = len(vocab)  # 词汇总数

/var/folders/9x/3n1chdsx4pxg5ymbp6p9ptwr0000gn/T/ipykernel_93134/911862339.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['processed'] = train['short_description'].apply(preprocess)


In [116]:
from onlineldavb import OnlineLDA

In [117]:

# 设置 LDA 模型的参数
K = 100  # 主题数
batchsize = 100  # 每次迭代处理的文档数
alpha = 50/K  # 主题权重的先验
eta = 0.1 # 词-主题分布的先验
tau_0 = 64  # 学习率控制参数
kappa = 0.7  # 学习率
vocab_dict = {word: index for index, word in enumerate(vectorizer.get_feature_names_out())}
# 初始化在线 LDA 模型
olda = OnlineLDA(vocab_dict, K, D, alpha, eta, tau_0, kappa)

# 训练模型
# 假设 doc_word_matrix 是一个 D x W 的矩阵，其中每一行代表一个文档的词袋表示
for idx in range(0, D, batchsize):
    batch = doc_word_matrix[idx:min(idx+batchsize, D)]
    olda.update_lambda(batch)
    if (idx + batchsize) % 1000 == 0:  # 每1000篇文档保存一次模型
        np.savetxt('lambda_iter_{}.dat'.format(idx + batchsize), olda._lambda)

In [118]:
from onlineldavb import dirichlet_expectation
import numpy as np
test['processed'] = test['short_description'].apply(preprocess)
test_doc_word_matrix = vectorizer.transform(test['processed']).toarray()


/var/folders/9x/3n1chdsx4pxg5ymbp6p9ptwr0000gn/T/ipykernel_93134/4190965496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['processed'] = test['short_description'].apply(preprocess)


In [119]:
def compute_perplexity(model, test_doc_word_matrix):
    Elogbeta = dirichlet_expectation(model._lambda)
    expElogbeta = np.exp(Elogbeta)

    total_log_likelihood = 0
    total_words = 0

    for d in range(len(test_doc_word_matrix)):
        doc = test_doc_word_matrix[d]
        ids = np.where(doc > 0)[0]
        if len(ids) == 0:
            continue
        cts = doc[ids]

        gamma = model._alpha + expElogbeta[:, ids].dot(cts)
        Elogtheta = dirichlet_expectation(gamma)
        expElogtheta = np.exp(Elogtheta)

        # 计算对数似然
        phinorm = np.dot(expElogtheta, expElogbeta[:, ids]) + 1e-100  # 避免除以零
        log_likelihood = np.sum(cts * np.log(phinorm / np.sum(phinorm)))  # 注意这里的变化

        total_log_likelihood += log_likelihood
        total_words += np.sum(cts)

    if total_words == 0:
        return float('inf')  # 如果没有词汇，返回无穷大表示困惑度无法计算

    perplexity = np.exp(-total_log_likelihood / total_words)
    return perplexity


In [120]:
perplexity = compute_perplexity(olda, test_doc_word_matrix)
print("Perplexity:", perplexity)

Perplexity: 6.5776479920480435
